# Data Preparation and Customer Analytics
# Task 1

Conduct analysis on client's transaction dataset and identify customer purchasing behaviours to generate insights and provide commercial recommendations.

## Work FLow

**1. Examine transaction data;** 

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.
    
    If anomalies are determined, make necessary changes and save it
    
**2. Examine customer data;**

    - Look for inconsistencies
    - missing data across the dataset
    - outliers
    - correctly identify category items
    - numeric data across all tables.

After completing the above tasks, merge the two datasets so it is ready for analysis.

**3. Data analysis and customer segments;**

    - define metrics
    - look at total sales
    - drivers of sales
    - where the highest sales are coming from
    - create charts and graphs
    - note interesting trends and insights
    
**4. Deep dive into customer segments;**

    - define recommendation from insights
    - determine which segments should be targeted
    - form overall conclusion based on analysis
    
**4. Save analysis**

    - save analysis and visuals

We will start this analysis by loading required libraries for analysis

In [1]:
# load libriaries
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline